In [1]:
import requests as r
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
alpha_token = os.getenv("ALPHA")
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle
import json
from database.comet import Comet
from processor.processor import Processor as p
from time import sleep
from extractor.tiingo_extractor import TiingoExtractor as tex

In [2]:
comet = Comet()

In [2]:
## manual loads
# cryptos = pd.read_csv("./data/digital_currency_list.csv")
# symbols = ["BTC","ETH","SHIB","NFTBS","CAKE","XTZ","DOGE","SAFEMOON","ADA","RARI","XRP","LUNA","USDT","ICP","AVAX","LTC","DOT","BUSD"]
# full_symbol_list = cryptos[cryptos["currency code"].isin(symbols)]
# comet.connect()
# comet.store("crypto7",full_symbol_list)
# comet.disconnect()

In [3]:
comet.connect()
c7 = comet.retrieve("crypto7")
comet.disconnect()

In [4]:
symbols = [
#             "XRP","LUNA","USDT","ICP","AVAX","LTC","DOT","BUSD",
          "MATIC"]

In [20]:
## Getting Symbols
start = datetime(2015,1,1).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)
comet.connect()
for crypto in symbols:
    try:
        stuff = tex.crypto_daily(crypto,start,end)
        final = pd.DataFrame(stuff.json()[0]["priceData"])
        final = p.column_date_processing(final)
        final["crypto"] = crypto
        comet.store("tiingo_prices",final)
    except Exception as e:
        print(str(e))
comet.disconnect()

2015-01-01 2022-01-09


In [5]:
## Getting Symbols
start = datetime(2015,1,1).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)
comet.connect()
for crypto in c7["currency code"][:1]:
    try:
        stuff = tex.crypto_intraday(crypto,start,end)
        final = pd.DataFrame(stuff.json()[0]["priceData"])
        final = p.column_date_processing(final)
        final["crypto"] = crypto
#         comet.store("tiingo_prices",final)
    except Exception as e:
        print(str(e))
comet.disconnect()

2015-01-01 2022-01-15
list index out of range


In [7]:
## Getting Symbols
comet.connect()
for crypto in symbols:
    try:
        params =  {'apikey':alpha_token,
                  'symbol':crypto,
                  'market':'USD',
                   'outputsize':'full'
                  }
        url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY'
        response = r.get(url,params=params)
        data = response.json()
        stuff = pd.DataFrame(data['Time Series (Digital Currency Daily)']).T.reset_index().rename(columns={"index":"date"})
        stuff["crypto"] = crypto
        for column in stuff.columns:
            if "a." in column:
                new_name = column.split(" ")[1].strip()
                stuff.rename(columns={column:new_name},inplace=True)
                stuff[new_name] = [float(x) for x in stuff[new_name]]
            else:
                if "volume" in column:
                    stuff.rename(columns={column:"volume"},inplace=True)
                if "6." in column:
                    stuff.rename(columns={column:"market_cap"},inplace=True)
                if "b." in column:
                    stuff.drop(column,axis=1,inplace=True)
        stuff = p.column_date_processing(stuff)
        comet.store("alpha_prices",stuff)
        sleep(15)
    except Exception as e:
        print(str(e))
comet.disconnect()

In [64]:
## Getting Symbols
comet.connect()
for crypto in c7["currency code"][:1]:
    try:
        params =  {'apikey':alpha_token,
                  'symbol':crypto,
                   'interval':'5min',
                  'market':'USD',
                   'outputsize':'full'
                  }
        url = 'https://www.alphavantage.co/query?function=CRYPTO_INTRADAY'
        response = r.get(url,params=params)
        data = response.json()
#         stuff = pd.DataFrame(data['Time Series (Digital Currency Daily)']).T.reset_index().rename(columns={"index":"date"})
#         stuff["crypto"] = crypto
#         for column in stuff.columns:
#             if "a." in column:
#                 new_name = column.split(" ")[1].strip()
#                 stuff.rename(columns={column:new_name},inplace=True)
#                 stuff[new_name] = [float(x) for x in stuff[new_name]]
#             else:
#                 if "volume" in column:
#                     stuff.rename(columns={column:"volume"},inplace=True)
#                 if "6." in column:
#                     stuff.rename(columns={column:"market_cap"},inplace=True)
#                 if "b." in column:
#                     stuff.drop(column,axis=1,inplace=True)
#         stuff = p.column_date_processing(stuff)
#         comet.store("intraday_prices",stuff)
#         sleep(15)
    except Exception as e:
        print(str(e))
comet.disconnect()

In [65]:
data

{'Meta Data': {'1. Information': 'Crypto Intraday (5min) Time Series',
  '2. Digital Currency Code': 'ADA',
  '3. Digital Currency Name': 'Cardano',
  '4. Market Code': 'USD',
  '5. Market Name': 'United States Dollar',
  '6. Last Refreshed': '2022-01-09 10:40:00',
  '7. Interval': '5min',
  '8. Output Size': 'Full size',
  '9. Time Zone': 'UTC'},
 'Time Series Crypto (5min)': {'2022-01-09 10:40:00': {'1. open': '1.16600',
   '2. high': '1.16900',
   '3. low': '1.16400',
   '4. close': '1.16900',
   '5. volume': 485622},
  '2022-01-09 10:35:00': {'1. open': '1.16000',
   '2. high': '1.16700',
   '3. low': '1.15900',
   '4. close': '1.16600',
   '5. volume': 779916},
  '2022-01-09 10:30:00': {'1. open': '1.15900',
   '2. high': '1.16000',
   '3. low': '1.15800',
   '4. close': '1.15900',
   '5. volume': 374016},
  '2022-01-09 10:25:00': {'1. open': '1.15700',
   '2. high': '1.16000',
   '3. low': '1.15700',
   '4. close': '1.15900',
   '5. volume': 423227},
  '2022-01-09 10:20:00': {'1.

In [66]:
stuff = pd.DataFrame(data['Time Series Crypto (5min)']).T.reset_index().rename(columns={"index":"date"})
stuff["crypto"] = crypto
for column in stuff.columns:
    if "." in column:
        new_name = column.split(" ")[1].strip()
        stuff.rename(columns={column:new_name},inplace=True)
        stuff[new_name] = [float(x) for x in stuff[new_name]]
stuff = p.column_date_processing(stuff)
stuff

,date,open,high,low,close,volume,crypto
0,2022-01-09 10:40:00,1.166,1.169,1.164,1.169,485622.0,ADA
1,2022-01-09 10:35:00,1.160,1.167,1.159,1.166,779916.0,ADA
2,2022-01-09 10:30:00,1.159,1.160,1.158,1.159,374016.0,ADA
3,2022-01-09 10:25:00,1.157,1.160,1.157,1.159,423227.0,ADA
4,2022-01-09 10:20:00,1.158,1.159,1.154,1.156,334274.0,ADA
...,...,...,...,...,...,...,...
995,2022-01-05 23:45:00,1.242,1.243,1.237,1.238,631124.0,ADA
996,2022-01-05 23:40:00,1.239,1.242,1.237,1.241,579762.0,ADA
997,2022-01-05 23:35:00,1.236,1.240,1.234,1.240,376676.0,ADA
998,2022-01-05 23:30:00,1.242,1.242,1.235,1.237,600492.0,ADA
